***stats & comparaison between the models***

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Mounting G drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Get the resources**

In [ ]:
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/pd_fast_save/" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/Models/" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/utils.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/main.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/imports.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/class_report.py" .

!rm -rf sample_data/
!unzip Models/RF_Models.zip 
!unzip Models/NB_Models.zip 
!unzip Models/XGBoost_Models.zip

***Imports***

In [ ]:
from imports import *

Load the data

In [ ]:
train_df_raw = pd.read_feather('/content/pd_fast_save/train_df_raw', use_threads=True)
test_df_raw = pd.read_feather('/content/pd_fast_save/train_df_raw', use_threads=True)

Load the models data

In [ ]:
rf = dill.load(open("RF_Models/rf4.obj","rb"))
nb = dill.load(open("NB_Models/nb4.obj","rb"))
xgboost = dill.load(open("XGBoost_Models/XGBoost3.obj","rb"))


In [ ]:
print(rf,"\n",nb,"\n",xgboost)

In [ ]:
features4 = ['frame.time_epoch','frame.time_delta','frame.time_delta_displayed','frame.time_relative','frame.len','frame.cap_len','wlan.fc.subtype','wlan.fc.ds','wlan.duration','wlan.ra','wlan.da','wlan.ta','wlan.sa','wlan.seq','wlan.wep.iv','wlan.wep.icv','data.len','class']
train = train_df_raw[features4]
test = test_df_raw[features4]

process the data

In [ ]:
str_to_cat(train_df_raw)
apply_cats(test_df_raw,train_df_raw)
x_train,y_train,nas,mapper=proc_df(train, 'class',do_scale=True)
x_valid,y_valid,nas ,mapper= proc_df(test,'class', do_scale=True,mapper=mapper,na_dict=nas)

In [ ]:
import matplotlib.pyplot as plt
attacks=['Flooding','Impersonation','Injection','normal']

plot the ROC for multi-classes classification

In [ ]:
def plot_ROC_curve(model , n_classes , x_valid , y_valid , labels , fig_title , plot_classes = True, save = False):
  if model.__class__.__name__=="GaussianNB":
    model = GaussianNB()
    model.fit(x_train, y_train) 
  y_predict_proba = model.predict_proba(x_valid)
  # Compute ROC curve and ROC AUC for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  all_y_test_i = np.array([])
  all_y_predict_proba = np.array([])
  for i in range(n_classes):
      y_test_i = np.array(list(map(lambda x: 1 if x == i else 0, y_valid)))
      all_y_test_i = np.concatenate([all_y_test_i, y_test_i])
      all_y_predict_proba = np.concatenate([all_y_predict_proba, y_predict_proba[:, i]])
      fpr[i], tpr[i], _ = roc_curve(y_test_i, y_predict_proba[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])
  # Compute micro-average ROC curve and ROC area
  fpr["average"], tpr["average"], _ = roc_curve(all_y_test_i, all_y_predict_proba)
  roc_auc["average"] = auc(fpr["average"], tpr["average"])
  # Plot average ROC Curve
  plt.figure()
  plt.plot(fpr["average"], tpr["average"],
          label='Average ROC curve (area = {0:0.2f})'
                ''.format(roc_auc["average"]),  color='deeppink', linestyle=':', linewidth=4)
  if plot_classes:
    # Plot each individual ROC curve
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], lw=2, label='ROC curve of  {0} (area = {1:0.2f})'.format(labels[i], roc_auc[i]))

  plt.plot([0, 1], [0, 1], 'k--', lw=2)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title(fig_title)
  plt.legend(loc="lower right")
  plt.show()
  if save:
    plt.savefig(fig_title+'.png')
  return fpr["average"], tpr["average"],roc_auc["average"]

In [ ]:
# fpr1,fpr2,fpr3,tpr1,tpr2,tpr3,roc_auc1,roc_auc2,roc_auc3 = (dict(),)*9

In [ ]:
_ , _ , _ = plot_ROC_curve(rf,4,x_valid,y_valid,attacks,'multiple_roc_curve_RF',plot_classes = True, save = True)

In [ ]:
fpr1, tpr1, roc_auc1 = plot_ROC_curve(rf,4,x_valid,y_valid,attacks,'multiple_roc_curve_RF1',plot_classes = False, save = True)

In [ ]:
_ , _ , _ = plot_ROC_curve(nb,4,x_valid,y_valid,attacks,'multiple_roc_curve_NB',plot_classes = True, save = True)

In [ ]:
fpr2, tpr2, roc_auc2 = plot_ROC_curve(nb,4,x_valid,y_valid,attacks,'multiple_roc_curve_NB1',plot_classes = False, save = True)

In [ ]:
_ , _ , _ =plot_ROC_curve(xgboost,4,x_valid,y_valid,attacks,'multiple_roc_curve_XGBoost',plot_classes = True, save = True)

In [ ]:
fpr3, tpr3, roc_auc3 = plot_ROC_curve(xgboost,4,x_valid,y_valid,attacks,'multiple_roc_curve_XGBoost1',plot_classes = False, save = True)

In [ ]:
def roc_comp(labels,fpr,tpr,roc_auc,fig_title,save=False):
  plt.figure()
  # Plot each individual ROC curve
  for i in range(len(labels)):
    plt.plot(fpr[i], tpr[i], lw=2, label='ROC curve of  {0} (area = {1:0.2f})'.format(labels[i], roc_auc[i]))
  
  plt.plot([0, 1], [0, 1], 'k--', lw=2)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title(fig_title)
  plt.legend(loc="lower right")
  plt.show()
  if save:
    plt.savefig(fig_title+'.png')


In [ ]:
labels = ['RF','NB','XGBoost']
fig_title = "ROC of all the models"
roc_comp(labels,[fpr1,frp2,fpr3],[tpr1,tpr2,tpr3],[roc_auc1,roc_auc2,roc_auc3],fig_title,True)